In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/product-catalogue/data/processed/public/task_1_query-product_ranking/product_catalogue-v0.2.csv
/kaggle/input/classification-model/config.json
/kaggle/input/classification-model/training_args.bin
/kaggle/input/classification-model/__results__.html
/kaggle/input/classification-model/tokenizer.json
/kaggle/input/classification-model/tokenizer_config.json
/kaggle/input/classification-model/pytorch_model.bin
/kaggle/input/classification-model/__notebook__.ipynb
/kaggle/input/classification-model/special_tokens_map.json
/kaggle/input/classification-model/__output__.json
/kaggle/input/classification-model/vocab.txt
/kaggle/input/classification-model/custom.css
/kaggle/input/classification-model/checkpoint-234522/config.json
/kaggle/input/classification-model/checkpoint-234522/trainer_state.json
/kaggle/input/classification-model/checkpoint-234522/training_args.bin
/kaggle/input/classification-model/checkpoint-234522/tokenizer.json
/kaggle/input/classification-model/checkpoint-2

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.1 MB/s eta 0:00:00


In [4]:
from datasets import ClassLabel, Value, DatasetDict, Dataset

In [5]:
from transformers import AutoModel
checkpoint = '../input/classification-model'
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at ../input/classification-model were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model.save_pretrained('./')

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
checkpoint = '../input/classification-model'
model = AutoModelForSequenceClassification.from_pretrained('./', num_labels=50)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [9]:
for name, param in model.named_parameters():
    if name.startswith("bert"): # choose whatever you like here
        param.requires_grad = False

In [10]:
test = pd.read_csv('../input/test-ranking-public/data/processed/public/task_1_query-product_ranking/test_public-v0.2.csv')

In [11]:
test.head()

,query_id,query,query_locale,product_id
0,33777,!qscreen fence without holes,us,B000HMCA7W
1,33777,!qscreen fence without holes,us,B0019Y6TTC
2,33777,!qscreen fence without holes,us,B001OJXVKW
3,33777,!qscreen fence without holes,us,B0030E1WLE
4,33777,!qscreen fence without holes,us,B003VNKLIY


In [12]:
test['query'].value_counts()

iphone headphones                             85
firestick                                     79
apple earphones                               74
futon frames full size without mattress       70
carhartt double front pants for men           68
                                              ..
ダミーカートリッジ 7.62                                 8
creative sound blaster g6                      8
jam home made ピアス                              8
インナーソール 扁平足矯正                                  8
nutribiotic plain rice protein powder 1 lb     8
Name: query, Length: 7245, dtype: int64

In [13]:
prod_cat = pd.read_csv('../input/product-catalogue/data/processed/public/task_1_query-product_ranking/product_catalogue-v0.2.csv')

In [14]:
train_df = pd.read_csv('../input/train-ranking/data/processed/public/task_1_query-product_ranking/train-v0.2.csv')

In [15]:
query_prod_train = pd.merge(train_df, prod_cat, right_on=['product_id','product_locale'], left_on=['product_id', 'query_locale'], suffixes=('_train','_catlog'), how='left')

In [16]:
query_prod_train.head()

,query_id,query,query_locale,product_id,esci_label,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,0,!awnmower tires without rims,us,B00004RA3F,irrelevant,"American Lawn Mower Company 1204-14 14-Inch 4-Blade Push Reel Lawn Mower, Red",NaN,"4-blade reel with a cutting width of 14” and durable 8.5” polymer wheels for maximum maneuverability\nMechanical lawn mower with an adjustable blade height of 0.5”-1.75” with scissor action cut\nManual grass cutter blades made of heat-treated alloy steel that stays sharp for years\nEnvironment-friendly alternative to gas-powered lawn mowers which is pollution-free and noise-free; Low maintenance, lightweight and easy to use\nCapable of cutting grass up to 4 inches tall",American Lawn Mower Company,Red,us
1,0,!awnmower tires without rims,us,B0018TWDOI,exact,Oregon 72-107 Universal Wheel 7X150 Diamond Plastic,NaN,"7 x 1.50""\nID 1/2-Inch\nHub Is Offset Plastic\nHub Length Is 1 3/8-Inch\nTread Design: Diamond",Oregon,⭐️ Exclusive,us
2,0,!awnmower tires without rims,us,B005O5Y3QI,exact,Carlisle Turf Saver Lawn & Garden Tire - 15X6-6 A,NaN,"Fit Type: Vehicle Specific\nEnhanced durability with its broad-shouldered, puncture resistant construction",Carlisle Paddles,NaN,us
3,0,!awnmower tires without rims,us,B0089RA0HY,substitute,"Honda 44710-VG3-010 Front Wheels, (Set of 2)","Honda Front Wheels (Set of 2) PLEASE NOTE: Check your owner's manual to be sure 44710-VL0-L02ZB, 44710-VG3-000 is the correct replacement part for your mower model. Consult owner's manual for proper part number identification and proper installation.",Sold on Amazon,Honda,NaN,us
4,0,!awnmower tires without rims,us,B0089RNSNM,exact,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01ZE) Lawn Mower Rear Wheel Set of 2,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01ZE AND 42710-VG3-B00) Lawn Mower Rear Wheel Set of 2,"Set of 2 Honda OEM Rear Wheels\nReplaces 42710-VE2-M01ZE and 42710-VG3-B00\n8 inch in diameter\nFits: HRB216 HXA, HRB216 HXAA, HRB216 TDA, HRB216 TDAA, HRB216 TXA, HRB216 TXAA, HRB217 HXA, HRB217 TDA, HRR216 S3DA, HRR216 SDA, HRR216K2 SDAA, HRR216K2 TDAA, HRR216K2 TKAA, HRR216K3 TDA, HRR216K4 TDA, HRS216 SDA, HRS216K1 SDA, HRS216K1 SDAA, HRS216K2 SDAA, HRS216K3 SDAA, HRT216 S3DA, HRT216 SDA, HRT216K1 S3DA, HRT216K1 S3DA-A, HRT216K1 SDA, HRT216K1 SDA-A, HRT216K1 SDAA, HRT216K1 SDAA-A, HRT216K1 TDAA, HRT216K1 TDAA-A, HRT216K2 TDAA, HRZ216 TDAA\nGenuine OEM Honda Part",Honda,NaN,us


In [17]:
def get_query_input(row):
    query_info = str(row.get('query', '')) + ' ' + str(row.get('query_locale', ''))
    return query_info.strip()

d = {'query': 'hi hello',}
assert get_query_input(d) == 'hi hello'
d = {'query': 'hi hello', 'query_locale': 'en'}
assert get_query_input(d) == 'hi hello en'

In [18]:
query_prod_train.fillna('', inplace=True)
query_prod_train['query_input'] = query_prod_train.apply(lambda row: get_query_input(row), axis=1)

In [19]:
def get_product_input(row):
    product_columns = ['product_title', 'product_locale', 'product_description']
    product_info = ''
    for col in product_columns:
        product_info += str(row.get(col, '')) + ' '
    return product_info.strip()

d = {'product_title': 'hi hello', 'product_locale': 'en', 'product_description': 'nike'}
assert get_product_input(d) == 'hi hello en nike'

In [20]:
query_prod_train['product_input'] = query_prod_train.apply(lambda row: get_product_input(row), axis=1)

In [21]:
query_prod_train.head(4)

,query_id,query,query_locale,product_id,esci_label,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,query_input,product_input
0,0,!awnmower tires without rims,us,B00004RA3F,irrelevant,"American Lawn Mower Company 1204-14 14-Inch 4-Blade Push Reel Lawn Mower, Red",,"4-blade reel with a cutting width of 14” and durable 8.5” polymer wheels for maximum maneuverability\nMechanical lawn mower with an adjustable blade height of 0.5”-1.75” with scissor action cut\nManual grass cutter blades made of heat-treated alloy steel that stays sharp for years\nEnvironment-friendly alternative to gas-powered lawn mowers which is pollution-free and noise-free; Low maintenance, lightweight and easy to use\nCapable of cutting grass up to 4 inches tall",American Lawn Mower Company,Red,us,!awnmower tires without rims us,"American Lawn Mower Company 1204-14 14-Inch 4-Blade Push Reel Lawn Mower, Red us"
1,0,!awnmower tires without rims,us,B0018TWDOI,exact,Oregon 72-107 Universal Wheel 7X150 Diamond Plastic,,"7 x 1.50""\nID 1/2-Inch\nHub Is Offset Plastic\nHub Length Is 1 3/8-Inch\nTread Design: Diamond",Oregon,⭐️ Exclusive,us,!awnmower tires without rims us,Oregon 72-107 Universal Wheel 7X150 Diamond Plastic us
2,0,!awnmower tires without rims,us,B005O5Y3QI,exact,Carlisle Turf Saver Lawn & Garden Tire - 15X6-6 A,,"Fit Type: Vehicle Specific\nEnhanced durability with its broad-shouldered, puncture resistant construction",Carlisle Paddles,,us,!awnmower tires without rims us,Carlisle Turf Saver Lawn & Garden Tire - 15X6-6 A us
3,0,!awnmower tires without rims,us,B0089RA0HY,substitute,"Honda 44710-VG3-010 Front Wheels, (Set of 2)","Honda Front Wheels (Set of 2) PLEASE NOTE: Check your owner's manual to be sure 44710-VL0-L02ZB, 44710-VG3-000 is the correct replacement part for your mower model. Consult owner's manual for proper part number identification and proper installation.",Sold on Amazon,Honda,,us,!awnmower tires without rims us,"Honda 44710-VG3-010 Front Wheels, (Set of 2) us Honda Front Wheels (Set of 2) PLEASE NOTE: Check your owner's manual to be sure 44710-VL0-L02ZB, 44710-VG3-000 is the correct replacement part for your mower model. Consult owner's manual for proper part number identification and proper installation."


In [22]:
scores = {'exact': 1.0, 'substitute': 0.1, 'complement': 0.01, 'irrelevant': 0.0}

In [23]:
query_prod_train['score'] = query_prod_train['esci_label'].apply(lambda l: scores[l])

In [24]:
!rm ./*

In [25]:
final_train = []
for query_id, _df in query_prod_train.groupby(['query_id']):
    final_train.append(_df.sort_values(by='score', ascending=False))

In [26]:
final_train_df = pd.concat(final_train, ignore_index=True)

In [27]:
assert final_train_df.shape[0] == train_df.shape[0]
final_train_df.head()

,query_id,query,query_locale,product_id,esci_label,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,query_input,product_input,score
0,0,!awnmower tires without rims,us,B0018TWDOI,exact,Oregon 72-107 Universal Wheel 7X150 Diamond Plastic,,"7 x 1.50""\nID 1/2-Inch\nHub Is Offset Plastic\nHub Length Is 1 3/8-Inch\nTread Design: Diamond",Oregon,⭐️ Exclusive,us,!awnmower tires without rims us,Oregon 72-107 Universal Wheel 7X150 Diamond Plastic us,1.0
1,0,!awnmower tires without rims,us,B005O5Y3QI,exact,Carlisle Turf Saver Lawn & Garden Tire - 15X6-6 A,,"Fit Type: Vehicle Specific\nEnhanced durability with its broad-shouldered, puncture resistant construction",Carlisle Paddles,,us,!awnmower tires without rims us,Carlisle Turf Saver Lawn & Garden Tire - 15X6-6 A us,1.0
2,0,!awnmower tires without rims,us,B0089RNSNM,exact,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01ZE) Lawn Mower Rear Wheel Set of 2,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01ZE AND 42710-VG3-B00) Lawn Mower Rear Wheel Set of 2,"Set of 2 Honda OEM Rear Wheels\nReplaces 42710-VE2-M01ZE and 42710-VG3-B00\n8 inch in diameter\nFits: HRB216 HXA, HRB216 HXAA, HRB216 TDA, HRB216 TDAA, HRB216 TXA, HRB216 TXAA, HRB217 HXA, HRB217 TDA, HRR216 S3DA, HRR216 SDA, HRR216K2 SDAA, HRR216K2 TDAA, HRR216K2 TKAA, HRR216K3 TDA, HRR216K4 TDA, HRS216 SDA, HRS216K1 SDA, HRS216K1 SDAA, HRS216K2 SDAA, HRS216K3 SDAA, HRT216 S3DA, HRT216 SDA, HRT216K1 S3DA, HRT216K1 S3DA-A, HRT216K1 SDA, HRT216K1 SDA-A, HRT216K1 SDAA, HRT216K1 SDAA-A, HRT216K1 TDAA, HRT216K1 TDAA-A, HRT216K2 TDAA, HRZ216 TDAA\nGenuine OEM Honda Part",Honda,,us,!awnmower tires without rims us,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01ZE) Lawn Mower Rear Wheel Set of 2 us Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01ZE AND 42710-VG3-B00) Lawn Mower Rear Wheel Set of 2,1.0
3,0,!awnmower tires without rims,us,B06XX6BM2R,exact,"MARASTAR 21446-2PK 15x6.00-6"" Front Tire Assembly Replacement-Craftsman Mower, Pack of 2",,"Tire: 2 pack 15x6. 00-6 tube-type turf SAVER tread - pneumatic 4 Ply rating\nWheels: Solid dark gray steel, 3"" Centered hub, 3/4"" Sintered iron bushings\nLoad capacity/speed: 400 lbs. / not for highway speeds\nUse: replacement for Craftsman riding mowers with front tire size 15x6. 00-6\nLow hassle, low cost solution to repairing or replacing your Craftsman riding Mower tires",MARASTAR,,us,!awnmower tires without rims us,"MARASTAR 21446-2PK 15x6.00-6"" Front Tire Assembly Replacement-Craftsman Mower, Pack of 2 us",1.0
4,0,!awnmower tires without rims,us,B077QMNXTS,exact,"MaxAuto 2 Pcs 16x6.50-8 Lawn Mower Tire for Garden Tractors Ridings, 4PR, Tubeless","<br>Tire Specifications:<br> 1. Material: Rubber<br> 2. Tire Size: 16X6.50-8<br> 3. Load Range: Max. 620lbs@28psi<br> 4. Tire Type: Tubeless<br> 5. Rim Width: 5.375""<br> 6. SW: 165mm / 6.50""<br> 7. OD: 410mm / 16.14""<br> 8. Rating: 4 PLY<br> 9. Tread Depth: 7.1mm/ 0.28""<br> 10. Pattern: P332<br> <br>Feature:<br> 1. Tires ONLY, rims not included<br> 2. Easy to install and easy to use<br> 3. Act as a spare or replacement of the broken one<br> 4. A perfect combination of premium rubber material and sophisticated workmanship<br> 5. Turf Tech tread is made to give you better traction on grassy surfaces, especially on hills where the potential for slippage exists, without causing harm to the turf itself.<br> 6. This tire is a popular replacement for many kind of lawn mower tires, riding lawn mower tires, lawn tractor tires, garden tractor tires, garden tractor pulling tires, and more.<br><br>Package Includes:<br> 2 x Lawn & Garden Tires","Set of 2 16X6.50-8, 16x6.50x8, 16-6.50-8 Lawn Garden Mower Tractor Tire\nType: Tubeless; 4 Ply; Tire only - Wheel not included\nTurf Tech tread is made to give you better traction on grassy surfaces, especially on hills where the potential for slippage exists, without causing harm to the turf itself.\nThis tire is a popular replacement for many kind of lawn mower tires, riding

In [28]:
def get_torch_device():
    """
    Getter for an available pyTorch device.
    :return: CUDA-capable GPU if available, CPU otherwise
    """
    return torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [29]:
import numpy as np
import torch

PADDED_Y_VALUE = -1
DEFAULT_EPS = 1e-10


def ndcg(y_pred, y_true, ats=None, gain_function=lambda x: torch.pow(2, x) - 1, padding_indicator=PADDED_Y_VALUE,
         filler_value=1.0):
    """
    Normalized Discounted Cumulative Gain at k.

    Compute NDCG at ranks given by ats or at the maximum rank if ats is None.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param ats: optional list of ranks for NDCG evaluation, if None, maximum rank is used
    :param gain_function: callable, gain function for the ground truth labels, e.g. torch.pow(2, x) - 1
    :param padding_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :param filler_value: a filler NDCG value to use when there are no relevant items in listing
    :return: NDCG values for each slate and rank passed, shape [batch_size, len(ats)]
    """
    idcg = dcg(y_true, y_true, ats, gain_function, padding_indicator)
    ndcg_ = dcg(y_pred, y_true, ats, gain_function, padding_indicator) / idcg
    idcg_mask = idcg == 0
    ndcg_[idcg_mask] = filler_value  # if idcg == 0 , set ndcg to filler_value

    assert (ndcg_ < 0.0).sum() >= 0, "every ndcg should be non-negative"

    return ndcg_


def __apply_mask_and_get_true_sorted_by_preds(y_pred, y_true, padding_indicator=PADDED_Y_VALUE):
    mask = y_true == padding_indicator

    y_pred[mask] = float('-inf')
    y_true[mask] = 0.0

    _, indices = y_pred.sort(descending=True, dim=-1)
    return torch.gather(y_true, dim=1, index=indices)


def dcg(y_pred, y_true, ats=None, gain_function=lambda x: torch.pow(2, x) - 1, padding_indicator=PADDED_Y_VALUE):
    """
    Discounted Cumulative Gain at k.

    Compute DCG at ranks given by ats or at the maximum rank if ats is None.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param ats: optional list of ranks for DCG evaluation, if None, maximum rank is used
    :param gain_function: callable, gain function for the ground truth labels, e.g. torch.pow(2, x) - 1
    :param padding_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :return: DCG values for each slate and evaluation position, shape [batch_size, len(ats)]
    """
    y_true = y_true.clone()
    y_pred = y_pred.clone()

    actual_length = y_true.shape[1]

    if ats is None:
        ats = [actual_length]
    ats = [min(at, actual_length) for at in ats]

    true_sorted_by_preds = __apply_mask_and_get_true_sorted_by_preds(y_pred, y_true, padding_indicator)

    discounts = (torch.tensor(1) / torch.log2(torch.arange(true_sorted_by_preds.shape[1], dtype=torch.float) + 2.0)).to(
        device=true_sorted_by_preds.device)

    gains = gain_function(true_sorted_by_preds)

    discounted_gains = (gains * discounts)[:, :np.max(ats)]

    cum_dcg = torch.cumsum(discounted_gains, dim=1)

    ats_tensor = torch.tensor(ats, dtype=torch.long) - torch.tensor(1)

    dcg = cum_dcg[:, ats_tensor]

    return dcg


def mrr(y_pred, y_true, ats=None, padding_indicator=PADDED_Y_VALUE):
    """
    Mean Reciprocal Rank at k.

    Compute MRR at ranks given by ats or at the maximum rank if ats is None.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param ats: optional list of ranks for MRR evaluation, if None, maximum rank is used
    :param padding_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :return: MRR values for each slate and evaluation position, shape [batch_size, len(ats)]
    """
    y_true = y_true.clone()
    y_pred = y_pred.clone()

    if ats is None:
        ats = [y_true.shape[1]]

    true_sorted_by_preds = __apply_mask_and_get_true_sorted_by_preds(y_pred, y_true, padding_indicator)

    values, indices = torch.max(true_sorted_by_preds, dim=1)
    indices = indices.type_as(values).unsqueeze(dim=0).t().expand(len(y_true), len(ats))

    ats_rep = torch.tensor(data=ats, device=indices.device, dtype=torch.float32).expand(len(y_true), len(ats))

    within_at_mask = (indices < ats_rep).type(torch.float32)

    result = torch.tensor(1.0) / (indices + torch.tensor(1.0))

    zero_sum_mask = torch.sum(values) == 0.0
    result[zero_sum_mask] = 0.0

    result = result * within_at_mask

    return result

In [30]:
def sinkhorn_scaling(mat, mask=None, tol=1e-6, max_iter=50):
    """
    Sinkhorn scaling procedure.
    :param mat: a tensor of square matrices of shape N x M x M, where N is batch size
    :param mask: a tensor of masks of shape N x M
    :param tol: Sinkhorn scaling tolerance
    :param max_iter: maximum number of iterations of the Sinkhorn scaling
    :return: a tensor of (approximately) doubly stochastic matrices
    """
    if mask is not None:
        mat = mat.masked_fill(mask[:, None, :] | mask[:, :, None], 0.0)
        mat = mat.masked_fill(mask[:, None, :] & mask[:, :, None], 1.0)

    for _ in range(max_iter):
        mat = mat / mat.sum(dim=1, keepdim=True).clamp(min=DEFAULT_EPS)
        mat = mat / mat.sum(dim=2, keepdim=True).clamp(min=DEFAULT_EPS)

        if torch.max(torch.abs(mat.sum(dim=2) - 1.)) < tol and torch.max(torch.abs(mat.sum(dim=1) - 1.)) < tol:
            break

    if mask is not None:
        mat = mat.masked_fill(mask[:, None, :] | mask[:, :, None], 0.0)

    return mat


def deterministic_neural_sort(s, tau, mask):
    """
    Deterministic neural sort.
    Code taken from "Stochastic Optimization of Sorting Networks via Continuous Relaxations", ICLR 2019.
    Minor modifications applied to the original code (masking).
    :param s: values to sort, shape [batch_size, slate_length]
    :param tau: temperature for the final softmax function
    :param mask: mask indicating padded elements
    :return: approximate permutation matrices of shape [batch_size, slate_length, slate_length]
    """
    dev = get_torch_device()

    n = s.size()[1]
    one = torch.ones((n, 1), dtype=torch.float32, device=dev)
    s = s.masked_fill(mask[:, :, None], -1e8)
    A_s = torch.abs(s - s.permute(0, 2, 1))
    A_s = A_s.masked_fill(mask[:, :, None] | mask[:, None, :], 0.0)

    B = torch.matmul(A_s, torch.matmul(one, torch.transpose(one, 0, 1)))

    temp = [n - m + 1 - 2 * (torch.arange(n - m, device=dev) + 1) for m in mask.squeeze(-1).sum(dim=1)]
    temp = [t.type(torch.float32) for t in temp]
    temp = [torch.cat((t, torch.zeros(n - len(t), device=dev))) for t in temp]
    scaling = torch.stack(temp).type(torch.float32).to(dev)  # type: ignore

    s = s.masked_fill(mask[:, :, None], 0.0)
    C = torch.matmul(s, scaling.unsqueeze(-2))

    P_max = (C - B).permute(0, 2, 1)
    P_max = P_max.masked_fill(mask[:, :, None] | mask[:, None, :], -np.inf)
    P_max = P_max.masked_fill(mask[:, :, None] & mask[:, None, :], 1.0)
    sm = torch.nn.Softmax(-1)
    P_hat = sm(P_max / tau)
    return P_hat


In [31]:
def sample_gumbel(samples_shape, device, eps=1e-10) -> torch.Tensor:
    """
    Sampling from Gumbel distribution.
    Code taken from "Stochastic Optimization of Sorting Networks via Continuous Relaxations", ICLR 2019.
    Minor modifications applied to the original code (masking).
    :param samples_shape: shape of the output samples tensor
    :param device: device of the output samples tensor
    :param eps: epsilon for the logarithm function
    :return: Gumbel samples tensor of shape samples_shape
    """
    U = torch.rand(samples_shape, device=device)
    return -torch.log(-torch.log(U + eps) + eps)

def stochastic_neural_sort(s, n_samples, tau, mask, beta=1.0, log_scores=True, eps=1e-10):
    """
    Stochastic neural sort. Please note that memory complexity grows by factor n_samples.
    Code taken from "Stochastic Optimization of Sorting Networks via Continuous Relaxations", ICLR 2019.
    Minor modifications applied to the original code (masking).
    :param s: values to sort, shape [batch_size, slate_length]
    :param n_samples: number of samples (approximations) for each permutation matrix
    :param tau: temperature for the final softmax function
    :param mask: mask indicating padded elements
    :param beta: scale parameter for the Gumbel distribution
    :param log_scores: whether to apply the logarithm function to scores prior to Gumbel perturbation
    :param eps: epsilon for the logarithm function
    :return: approximate permutation matrices of shape [n_samples, batch_size, slate_length, slate_length]
    """
    dev = get_torch_device()

    batch_size = s.size()[0]
    n = s.size()[1]
    s_positive = s + torch.abs(s.min())
    samples = beta * sample_gumbel([n_samples, batch_size, n, 1], device=dev)
    if log_scores:
        s_positive = torch.log(s_positive + eps)

    s_perturb = (s_positive + samples).view(n_samples * batch_size, n, 1)
    mask_repeated = mask.repeat_interleave(n_samples, dim=0)

    P_hat = deterministic_neural_sort(s_perturb, tau, mask_repeated)
    P_hat = P_hat.view(n_samples, batch_size, n, n)
    return P_hat

In [32]:
def neuralNDCG(y_pred, y_true, padded_value_indicator=PADDED_Y_VALUE, temperature=1., powered_relevancies=True, k=None,
               stochastic=False, n_samples=32, beta=0.1, log_scores=True):
    """
    NeuralNDCG loss introduced in "NeuralNDCG: Direct Optimisation of a Ranking Metric via Differentiable
    Relaxation of Sorting" - https://arxiv.org/abs/2102.07831. Based on the NeuralSort algorithm.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :param temperature: temperature for the NeuralSort algorithm
    :param powered_relevancies: whether to apply 2^x - 1 gain function, x otherwise
    :param k: rank at which the loss is truncated
    :param stochastic: whether to calculate the stochastic variant
    :param n_samples: how many stochastic samples are taken, used if stochastic == True
    :param beta: beta parameter for NeuralSort algorithm, used if stochastic == True
    :param log_scores: log_scores parameter for NeuralSort algorithm, used if stochastic == True
    :return: loss value, a torch.Tensor
    """
    dev = get_torch_device()

    if k is None:
        k = y_true.shape[1]

    mask = (y_true == padded_value_indicator)
    # Choose the deterministic/stochastic variant
    if stochastic:
        P_hat = stochastic_neural_sort(y_pred.unsqueeze(-1), n_samples=n_samples, tau=temperature, mask=mask,
                                       beta=beta, log_scores=log_scores)
    else:
        P_hat = deterministic_neural_sort(y_pred.unsqueeze(-1), tau=temperature, mask=mask).unsqueeze(0)

    # Perform sinkhorn scaling to obtain doubly stochastic permutation matrices
    P_hat = sinkhorn_scaling(P_hat.view(P_hat.shape[0] * P_hat.shape[1], P_hat.shape[2], P_hat.shape[3]),
                             mask.repeat_interleave(P_hat.shape[0], dim=0), tol=1e-6, max_iter=50)
    P_hat = P_hat.view(int(P_hat.shape[0] / y_pred.shape[0]), y_pred.shape[0], P_hat.shape[1], P_hat.shape[2])

    # Mask P_hat and apply to true labels, ie approximately sort them
    P_hat = P_hat.masked_fill(mask[None, :, :, None] | mask[None, :, None, :], 0.)
    y_true_masked = y_true.masked_fill(mask, 0.).unsqueeze(-1).unsqueeze(0)
    if powered_relevancies:
        y_true_masked = torch.pow(2., y_true_masked) - 1.

    ground_truth = torch.matmul(P_hat, y_true_masked).squeeze(-1)
    discounts = (torch.tensor(1.) / torch.log2(torch.arange(y_true.shape[-1], dtype=torch.float) + 2.)).to(dev)
    discounted_gains = ground_truth * discounts

    if powered_relevancies:
        idcg = dcg(y_true, y_true, ats=[k]).permute(1, 0)
    else:
        idcg = dcg(y_true, y_true, ats=[k], gain_function=lambda x: x).permute(1, 0)

    discounted_gains = discounted_gains[:, :, :k]
    ndcg = discounted_gains.sum(dim=-1) / (idcg + DEFAULT_EPS)
    idcg_mask = idcg == 0.
    ndcg = ndcg.masked_fill(idcg_mask.repeat(ndcg.shape[0], 1), 0.)

    assert (ndcg < 0.).sum() >= 0, "every ndcg should be non-negative"
    if idcg_mask.all():
        return torch.tensor(0.)

    mean_ndcg = ndcg.sum() / ((~idcg_mask).sum() * ndcg.shape[0])  # type: ignore
    return -1. * mean_ndcg  # -1 cause we want to maximize NDCG

In [33]:
y_pred = torch.tensor([[0.1,0.2,0.7],
                       [0.1,0.2,0.7],
                       [0.1,0.2,0.7]])

In [34]:
y_true = torch.tensor([[0.01,0.1,1],
                       [0.01,0.1,1],
                       [0.01,0.1,1]], dtype=torch.float)

In [35]:
#neuralNDCG(y_pred=y_pred, y_true=y_true) # tensor(-0.6609)

In [36]:
final_train_df.drop(columns=['query', 'esci_label',
       'product_title', 'product_description', 'product_bullet_point',
       'product_brand', 'product_color_name', 'product_locale'], inplace=True)

In [37]:
final_train_df.head(1)

,query_id,query_locale,product_id,query_input,product_input,score
0,0,us,B0018TWDOI,!awnmower tires without rims us,Oregon 72-107 Universal Wheel 7X150 Diamond Plastic us,1.0


In [38]:
from sklearn.model_selection import train_test_split
train = []
test = [] 
split = 0.8
for query_locale, _df in final_train_df.groupby(['query_locale']):
    ngroups = len(_df.groupby('query_id').groups)
    cur_grp = 0
    for query_id, __df in _df.groupby(['query_id']):
        cur_grp += 1
        if __df.shape[0] < 50:
            extra_df = __df.sample(frac=(50//__df.shape[0]) + 1.0, replace=True, random_state=17)
        if cur_grp <= split * ngroups:
            train.append(__df.sample(n=min(50, __df.shape[0]), random_state=17))
            if __df.shape[0] < 50:
                train.append(extra_df.sample(n=50-__df.shape[0], random_state=17))
        else:
            test.append(__df.sample(n=min(50, __df.shape[0]), random_state=17))
            if __df.shape[0] < 50:
                test.append(extra_df.sample(n=50-__df.shape[0], random_state=17))

In [39]:
train_df = pd.concat(train, ignore_index=True)
test_df = pd.concat(test, ignore_index=True)

In [40]:
train_df.shape, test_df.shape

((1352100, 6), (338100, 6))

In [41]:
train_df.query_locale.value_counts(normalize=True)

us    0.617928
jp    0.215480
es    0.166593
Name: query_locale, dtype: float64

In [42]:
test_df.query_locale.value_counts(normalize=True)

us    0.617865
jp    0.215469
es    0.166667
Name: query_locale, dtype: float64

In [43]:
# convert to query_input [SEP] prod_input1 [SEP] prod_input2 ... [SEP] prod_inputn
def ranking_format(df):
    train = {}
    train['input'] = []
    train['labels'] = []
    for query, _df in df.groupby(['query_input']):
        input_str = query + '[SEP] '
        scores = _df.score.to_list()
        inputs = _df.product_input.to_list()
        for i in inputs:
            input_str += i + ' [SEP] '
        train['input'].append(input_str.strip())
        train['labels'].append(scores)
    return pd.DataFrame.from_dict(train,orient='index').transpose()
            

In [44]:
train_df.head(1)

,query_id,query_locale,product_id,query_input,product_input,score
0,1,es,B08NDS1HS2,!solid camiseta sin manga es,Zereesa Camiseta sin Mangas de Punto con Cuello en V para Mujer Chaleco de Punto sin Mangas Camiseta sin Mangas de Punto Informal de Color sólido Chaleco de Punto Informal Chaleco de Trabajo S-XXL es,1.0


In [45]:
compr_train_df = ranking_format(train_df)
compr_test_df = ranking_format(test_df)

In [46]:
compr_train_df.shape, compr_test_df.shape

((27042, 2), (6762, 2))

In [47]:
from transformers import AutoTokenizer,DataCollatorWithPadding
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

In [48]:
from datasets import ClassLabel, Value, DatasetDict, Dataset
dataset = DatasetDict({
        'train': Dataset.from_pandas(compr_train_df),
        'test': Dataset.from_pandas(compr_test_df),
    }   
)

In [49]:
def tokenize_label_map(batch):
    tokens = tokenizer(batch['input'], max_length=512, padding='max_length', truncation=True)
    tokens['labels'] = batch['labels']
    return tokens
encoded_dataset = dataset.map(tokenize_label_map, batched=True)

  0%|          | 0/28 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [50]:
tokenized_datasets = encoded_dataset
tokenized_datasets.set_format("torch")

In [51]:
tokenized_datasets['train']['labels'][0].shape

torch.Size([50])

In [52]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [53]:
import numpy as np
def eval_dcg(y_pred, y_true, ats=None, gain_function=lambda x: torch.pow(2, x) - 1, padding_indicator=PADDED_Y_VALUE):
    """
    Discounted Cumulative Gain at k.

    Compute DCG at ranks given by ats or at the maximum rank if ats is None.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param ats: optional list of ranks for DCG evaluation, if None, maximum rank is used
    :param gain_function: callable, gain function for the ground truth labels, e.g. torch.pow(2, x) - 1
    :param padding_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :return: DCG values for each slate and evaluation position, shape [batch_size, len(ats)]
    """
    y_true = y_true.clone()
    y_pred = y_pred.clone()

    actual_length = y_true.shape[1]

    if ats is None:
        ats = [actual_length]
    ats = [min(at, actual_length) for at in ats]

    true_sorted_by_preds = __apply_mask_and_get_true_sorted_by_preds(y_pred, y_true, padding_indicator)

    discounts = (torch.tensor(1) / torch.log2(torch.arange(true_sorted_by_preds.shape[1], dtype=torch.float) + 2.0)).to(
        device=true_sorted_by_preds.device)

    gains = gain_function(true_sorted_by_preds)

    discounted_gains = (gains * discounts)[:, :np.max(ats)]

    cum_dcg = torch.cumsum(discounted_gains, dim=1)

    ats_tensor = torch.tensor(ats, dtype=torch.long) - torch.tensor(1)

    dcg = cum_dcg[:, ats_tensor]

    return dcg
def ai_ndcg(y_pred, y_true, ats=None, gain_function=lambda x: torch.pow(2, x) - 1, padding_indicator=PADDED_Y_VALUE,
         filler_value=1.0):
    """
    Normalized Discounted Cumulative Gain at k.
    Compute NDCG at ranks given by ats or at the maximum rank if ats is None.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param ats: optional list of ranks for NDCG evaluation, if None, maximum rank is used
    :param gain_function: callable, gain function for the ground truth labels, e.g. torch.pow(2, x) - 1
    :param padding_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :param filler_value: a filler NDCG value to use when there are no relevant items in listing
    :return: NDCG values for each slate and rank passed, shape [batch_size, len(ats)]
    """
    idcg = eval_dcg(y_true, y_true, ats, gain_function, padding_indicator)
    ndcg_ = eval_dcg(y_pred, y_true, ats, gain_function, padding_indicator) / idcg
    idcg_mask = idcg == 0
    ndcg_[idcg_mask] = filler_value  # if idcg == 0 , set ndcg to filler_value

    assert (ndcg_ < 0.0).sum() >= 0, "every ndcg should be non-negative"

    return ndcg_

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    metrics_dict = {
        'ndcg': torch.mean(ai_ndcg(y_pred=torch.from_numpy(logits), y_true=torch.from_numpy(labels))).item(),
    }
    return metrics_dict

In [54]:
from transformers import Trainer
from transformers import TrainingArguments
import torch

class RankingTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        loss = torch.mean(neuralNDCG(y_pred=logits, y_true=labels))
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(evaluation_strategy="epoch", report_to="none", output_dir = '/kaggle/working/', load_best_model_at_end=True, save_total_limit=2, save_strategy='epoch', logging_strategy='steps', logging_steps=50, log_level='info', log_level_replica='info', fp16=False, learning_rate=1e-4)

trainer = RankingTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],#.select([0,1,2,3,4,5,6,7,8,9]),
    eval_dataset=tokenized_datasets["test"],#.select([0,1,2,3,4,5,6,7,8,9]),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [55]:
import traceback
try:
    trainer.train()
except:
    print(traceback.format_exc())
    print("Training stopped")

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input. If input are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27042
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10143


Epoch,Training Loss,Validation Loss,Ndcg
1,-0.908100,-0.924399,0.925912
2,-0.910000,-0.924751,0.925495
3,-0.908300,-0.924957,0.926026


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input. If input are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6762
  Batch size = 8
Saving model checkpoint to /kaggle/working/checkpoint-3381
Configuration saved in /kaggle/working/checkpoint-3381/config.json
Model weights saved in /kaggle/working/checkpoint-3381/pytorch_model.bin
tokenizer config file saved in /kaggle/working/checkpoint-3381/tokenizer_config.json
Special tokens file saved in /kaggle/working/checkpoint-3381/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input. If input are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num ex

In [56]:
try:
    trainer.save_model('/kaggle/working/')
except:
    print("Trainer save stopped")

Saving model checkpoint to /kaggle/working/
Configuration saved in /kaggle/working/config.json
Model weights saved in /kaggle/working/pytorch_model.bin
tokenizer config file saved in /kaggle/working/tokenizer_config.json
Special tokens file saved in /kaggle/working/special_tokens_map.json
